# COURSERA IBM Capstone - Week I

## Introduction

What is the best price range for your new restaurant? What if I told you that this study will surely be able to answer to your question.
We are going to check the food venues in the center of the major cities of Italy, and build a model on their foursquare data.
We will use their rating, food type and position of the city and within the city to create a model that will predict the best price range for your new business!!
Trust me, I'm an Engeneer*

First we are going to import the needed libraries

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import requests
from bs4 import BeautifulSoup

The table of the major Italian city is imported from wikipedia, then sorted, polished and finally all the city with more than 200.000 inhabitants are selected.

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_cities_in_Italy'
html = requests.get(wiki_url).text
soup = BeautifulSoup(html, 'lxml')

In [3]:
my_table = soup.find('table', {'class' : 'wikitable sortable'})

In [4]:
Rank = []
City = []
Population = []
Estimate = []
Change = []
Region = []
for row in my_table.find_all('tr'):
    cells = row.find_all('td')
    if len(cells) == 6:
        Rank.append(cells[0].find(text = True))
        City.append(cells[1].find(text = True))
        Population.append(cells[2].find(text = True))
        Estimate.append(cells[3].find(text = True))
        Change.append(cells[4].find(text = True))
        Region.append(cells[5].find(text = True))


for i in Estimate:
    j = i.strip()
    Estimate[Estimate.index(i)] = j
for i in Estimate:
    i = str(i)
    j = i.replace(',', '')
    Estimate[Estimate.index(i)] = int(j)

In [5]:
Italy_df = pd.DataFrame({'Rank' : Rank, 'City': City, 'Population' : Population, 'Estimate' : Estimate,
                         'Change' : Change, 'Region' : Region})
Italy_df = Italy_df.drop(['Rank', 'Population', 'Change', 'Region'], axis = 1)
Italy_df = Italy_df[Italy_df.Estimate > 200000]

In [6]:
Latitude = []
Longitude = []

for i in Italy_df.City:
    address = i + ', Italy'
    geolocator = Nominatim(user_agent = 'my-application')
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    Latitude.append(latitude)
    Longitude.append(longitude)
    
Italy_df['Latitude'] = Latitude
Italy_df['Longitude'] = Longitude

In [7]:
Italy_df

,City,Estimate,Latitude,Longitude
0,Rome,2872800,41.894802,12.485338
1,Milan,1366180,45.466797,9.190498
2,Naples,966144,40.835934,14.248783
3,Turin,882523,45.067755,7.682489
4,Palermo,668405,38.111227,13.352443
5,Genoa,580097,44.407260,8.933862
6,Bologna,389261,44.493671,11.343035
7,Florence,380948,43.769871,11.255576
8,Bari,323370,41.125784,16.862029
9,Catania,311620,37.502235,15.087380


Now we are going to take a view of one the cities

In [34]:
map_italy = folium.Map(location=[42.504154, 12.646361], zoom_start = 6)

for lat, lng, city in zip(Italy_df['Latitude'], Italy_df['Longitude'], Italy_df['City']):
    label = '{}, Italy'.format(city)
    label = folium.Popup(label, parse_html = True)
    folium.Circle(
        [lat, lng],
        radius = 2000,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_italy)

map_italy

With this function we are going to import the 33 food venues closest to the center of the city. Unfortunately we cannot put up the maximum number possible (50) for limit in foursquare accout premium calls (500 max per day).

In [9]:
CLIENT_ID = 'WFU4QVT5P2VFOGGHWIT1N2IFDKK15VR2LNBUXTH4GJ1JSMBQ'
CLIENT_SECRET = 'E5HD1RAH4PVBJ403KLILAI0BU4MY0ZMRYEBRKR0EIBBZBGVE'
VERSION = '20180605'


def getNearbyVenues(names, latitudes, longitudes, radius = 2000, LIMIT = 33):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()['response']
        for v in results['venues'] :
            venues_list.append([(name,
                                 lat,
                                 lng,
                                 v['name'],
                                 v['id'],
                                 v['location']['lat'],
                                 v['location']['lng'], 
                                 v['categories'][0]['name'])])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City',
                             'City Latitude',
                             'City Longitude', 
                             'Venue',
                             'Venue Id',
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

Here we apply the function to the city venues and we get their name, the venue Id, spoacial coordinates and category.

In [10]:
Italy_venues = getNearbyVenues(names = Italy_df['City'],
                                 latitudes = Italy_df['Latitude'],
                                 longitudes = Italy_df['Longitude'])

Rome
Milan
Naples
Turin
Palermo
Genoa
Bologna
Florence
Bari
Catania
Venice
Verona
Messina
Padua
Trieste


In [11]:
Italy_venues.shape

(495, 8)

With this function we use the venue Id to access to more information abou the venue itself, the price and the rating.

In [12]:
CLIENT_ID = 'WFU4QVT5P2VFOGGHWIT1N2IFDKK15VR2LNBUXTH4GJ1JSMBQ'
CLIENT_SECRET = 'E5HD1RAH4PVBJ403KLILAI0BU4MY0ZMRYEBRKR0EIBBZBGVE'
VERSION = '20180605'

def PriceandRating (v_id):
    ratings_list=[]
    for i in v_id:
        venue_id = i
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
        results = requests.get(url).json()['response']
        try:
            ratings_list.append([(i,
                                  results['venue']['rating'],
                                  results['venue']['price']['tier'])])
        except:
            ratings_list.append([(i,
                                  'Rating or Tier not present',
                                  'Rating or Tier not present')])
        
    rating_venues = pd.DataFrame([item for ratings_list in ratings_list for item in ratings_list])
    rating_venues.columns = ['Venue Id',
                             'Venue Rating', 
                             'Venue Price']
    
    return(rating_venues)

Now we apply the function and we merge the two dataframes

In [13]:
Venue_ratings = PriceandRating(Italy_venues['Venue Id'])

In [29]:
#Venue_ratings_2 = Venue_ratings.set_index('Venue Id')
#Italy_venues = Italy_venues.set_index('Venue Id')
Italy_venues_data = Italy_venues.join(Venue_ratings_2)
Italy_venues_data = Italy_venues_data[Italy_venues_data['Venue Rating'] != 'Rating or Tier not present']
Italy_venues_data = Italy_venues_data.reset_index(drop = True)

In [30]:
Italy_venues_data.head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Rating,Venue Price
0,Rome,41.894802,12.485338,"Angelino ""ai Fori"" dal 1947",41.893154,12.487968,Italian Restaurant,8.3,3
1,Rome,41.894802,12.485338,Antico Forno Roscioli,41.894624,12.474327,Bakery,9.3,2
2,Rome,41.894802,12.485338,Panella,41.894090,12.500857,Café,8.1,3
3,Rome,41.894802,12.485338,Tazza d'Oro,41.899435,12.477359,Coffee Shop,9.1,1
4,Rome,41.894802,12.485338,Antico Caffè Greco,41.905575,12.481656,Café,8.7,1


A quick plot of the venues selected

In [33]:
map_venues = folium.Map(location=[42.504154, 12.646361], zoom_start = 6)

for lat, lng, venue, cat in zip(Italy_venues_data['Venue Latitude'],
                                Italy_venues_data['Venue Longitude'],
                                Italy_venues_data['Venue'],
                                Italy_venues_data['Venue Category']):
    label = '{}, {}'.format(venue, cat)
    label = folium.Popup(label, parse_html = True)
    folium.Marker(
        [lat, lng],
        popup = label).add_to(map_venues)

map_venues

We prepare the dataframe for the data analysis one hot encoding the non numerical variables and dropping the ones that we are not going to use in the creation of the model

In [35]:
Italy_venues_data = pd.get_dummies(Italy_venues_data, columns=['City', 'Venue Category'], drop_first=True)
Italy_venues_data.head()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Rating,Venue Price,City_Bologna,City_Catania,City_Florence,City_Genoa,City_Messina,City_Milan,City_Naples,City_Padua,City_Palermo,City_Rome,City_Trieste,City_Turin,City_Venice,City_Verona,Venue Category_Bakery,Venue Category_Bar,Venue Category_Bistro,Venue Category_Breakfast Spot,Venue Category_Burger Joint,Venue Category_Café,Venue Category_Chinese Restaurant,Venue Category_Cocktail Bar,Venue Category_Coffee Shop,Venue Category_Cupcake Shop,Venue Category_Deli / Bodega,Venue Category_Dessert Shop,Venue Category_Diner,Venue Category_Emilia Restaurant,Venue Category_Fast Food Restaurant,Venue Category_Food Stand,Venue Category_Gastropub,Venue Category_Ice Cream Shop,Venue Category_Indian Restaurant,Venue Category_Italian Restaurant,Venue Category_Japanese Restaurant,Venue Category_Juice Bar,Venue Category_Mediterranean Restaurant,Venue Category_Pizza Place,Venue Category_Pub,Venue Category_Restaurant,Venue Category_Sandwich Place,Venue Category_Seafood Restaurant,Venue Category_Snack Place,Venue Category_Steakhouse,Venue Category_Sushi Restaurant,Venue Category_Trattoria/Osteria,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Wine Bar
0,41.894802,12.485338,"Angelino ""ai Fori"" dal 1947",41.893154,12.487968,8.3,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,41.894802,12.485338,Antico Forno Roscioli,41.894624,12.474327,9.3,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,41.894802,12.485338,Panella,41.894090,12.500857,8.1,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,41.894802,12.485338,Tazza d'Oro,41.899435,12.477359,9.1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,41.894802,12.485338,Antico Caffè Greco,41.905575,12.481656,8.7,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
Venues_dataset_2 = Italy_venues_data.drop(['Venue', 'City Latitude', 'City Longitude'], axis = 1)
Venues_dataset_2.columns.name = Venues_dataset_2.index.name
Venues_dataset_2.index.name = None

In [37]:
Venues_dataset_2.head()

,Venue Latitude,Venue Longitude,Venue Rating,Venue Price,City_Bologna,City_Catania,City_Florence,City_Genoa,City_Messina,City_Milan,City_Naples,City_Padua,City_Palermo,City_Rome,City_Trieste,City_Turin,City_Venice,City_Verona,Venue Category_Bakery,Venue Category_Bar,Venue Category_Bistro,Venue Category_Breakfast Spot,Venue Category_Burger Joint,Venue Category_Café,Venue Category_Chinese Restaurant,Venue Category_Cocktail Bar,Venue Category_Coffee Shop,Venue Category_Cupcake Shop,Venue Category_Deli / Bodega,Venue Category_Dessert Shop,Venue Category_Diner,Venue Category_Emilia Restaurant,Venue Category_Fast Food Restaurant,Venue Category_Food Stand,Venue Category_Gastropub,Venue Category_Ice Cream Shop,Venue Category_Indian Restaurant,Venue Category_Italian Restaurant,Venue Category_Japanese Restaurant,Venue Category_Juice Bar,Venue Category_Mediterranean Restaurant,Venue Category_Pizza Place,Venue Category_Pub,Venue Category_Restaurant,Venue Category_Sandwich Place,Venue Category_Seafood Restaurant,Venue Category_Snack Place,Venue Category_Steakhouse,Venue Category_Sushi Restaurant,Venue Category_Trattoria/Osteria,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Wine Bar
0,41.893154,12.487968,8.3,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,41.894624,12.474327,9.3,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,41.894090,12.500857,8.1,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,41.899435,12.477359,9.1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,41.905575,12.481656,8.7,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Last we export the dataset as a csv to be used in the week 2 of the project

In [38]:
Venues_dataset_2.to_csv('Venues_Italy.csv')

*= this could not be true